In [21]:
import pandas as pd 
import numpy as np
import re

In [22]:
df = pd.read_csv('origindata/transfers.csv')
df['year'] = pd.to_datetime(df['transfer_date'], errors='coerce').dt.year

# 불필요 클럽 제거
df = df[~df['to_club_name'].isin(['Retired', 'Without Club', '011'])]

# 결측 market_value_in_eur → 0
df['market_value_in_eur'] = df['market_value_in_eur'].fillna(0)

In [23]:
# 클럽명 정규화 함수 (확장 버전)
def normalize_club_name(name: str) -> str:
    """‘U21’, ‘Res.’, ‘Youth’, ‘B’, ‘C’, ‘II’, ‘S17’ 등과
       ‘FC’·‘F.C.’ 접미어를 제거하여 모(母)클럽으로 통합"""
    if pd.isna(name):
        return name
    name = str(name).strip()

    # 접미어 패턴 제거 ─ Youth·U‑팀·Reserves·B·C·II·S17·S21 …
    pattern_suffix = re.compile(
        r"""
        \s+                                   # 앞 공백
        (?:                                   # 다음 중 하나
            U\d{1,2}        |                 # U17, U21 …
            S\d{1,2}        |                 # S17, S21 …
            Y\d{1,2}        |                 # Y17, Y21 …
            Res\.?          | Reserve[s]? |   # Res., Reserve(s)
            Youth | Yth\.?  |                 # Youth, Yth.
            B | C | II | Y.                   # B, C, II, Y
        )
        $                                     
        """,
        flags=re.IGNORECASE | re.VERBOSE
    )
    name = re.sub(pattern_suffix, '', name)

    # 끝의 ' FC' / ' F.C.' 제거
    name = re.sub(r'\s+F\.?C\.?$', '', name, flags=re.IGNORECASE)

    return name.strip()

In [24]:
df['club_clean'] = df['to_club_name'].apply(normalize_club_name)

# 7. 그룹 연산: 클린 클럽 × 연도
grouped = (
    df.groupby(['club_clean', 'year'], as_index=False)['market_value_in_eur']
      .sum()
)

# 8. 10 000€ 단위로 환산 & 반올림
grouped['value_10k'] = (grouped['market_value_in_eur'] / 10000).round(2)


In [25]:
# 9. 피벗: 행=클럽, 열=연도
pivot = (
    grouped
      .pivot(index='club_clean', columns='year', values='value_10k')
      .fillna(0)
      .sort_index()
)

# 10. 저장
pivot.to_csv('dataset/transfer_value_by_club_year.csv', encoding='utf-8-sig')

print(pivot.head())


year             1993  1994  1995  1996  1997  1998  1999  2000  2001  2002  \
club_clean                                                                    
011               0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1 Dezembro        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1. Göppinger SV   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1.FC Bocholt      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1.FC Düren        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

year             ...  2017  2018  2019  2020  2021  2022  2023  2024  2025  \
club_clean       ...                                                         
011              ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1 Dezembro       ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1. Göppinger SV  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1.FC Bocholt     ...   2.5   0.0   0.0   0.0   0.0   0.0

In [26]:
# 피벗 테이블 불러오기
pivot = pd.read_csv('dataset/transfer_value_by_club_year_cleaned.csv', index_col=0)
pivot['sum'] = pivot.sum(axis=1)

# 전체 합이 0인 클럽 제거
pivot = pivot[pivot['sum'] > 0]

# '순위' 열 추가 (sum 기준 내림차순 순위)
pivot['순위'] = pivot['sum'].rank(ascending=False, method='min').astype(int)

# 필요한 열만 선택
result = pivot[['sum', '순위']].copy()
result = result.reset_index().rename(columns={'club_clean': 'club'})

# 정렬 (순위 오름차순 = sum 내림차순)
result = result.sort_values(by='순위')

# 저장 및 확인
result.to_csv('dataset/transfer_value_confirmed.csv', index=False, encoding='utf-8-sig')
print(result.head())


          club       sum  순위
657    Chelsea  391000.0   1
1486  Juventus  259485.0   2
2128  Paris SG  258580.0   3
1790  Man City  249052.5   4
1791   Man Utd  232252.5   5
